## O Algoritmo

Para compreender como o _PageRank_ funciona, vamos examinar uma situação onde nossa rede é composta de apenas 4 sites: $A$, $B$, $C$ e $D$ e temos um hipotético usuário que caminha de um site para o outro sem preferências.
Uma característica dessa tecnologia é que a soma de todos os _PageRank_ é $1$. Portanto, inicialmente, todos tem _PageRank_ igual a $0,25$.
<!-- spaces -->
<img src="img/ABCD1.png" width="300" height="300"/>

Vamos supor agora que eles estejam conectados de alguma forma entre si. Para melhor visualização, as conexões de um site consigo mesmo e o caminho de duas vias foi oculto. Nessa primeira situação, os sites $B$, $C$ e $D$ resultam unicamente em $A$. Assim, o _PageRank_ de $A$ pode ser calculado como a soma do _PageRank_ dos demais sites:
<!-- spaces -->
<img src="img/ABCD2.png" width="300" height="300"/>
<!-- spaces -->
$$PR(A) = PR(B) + PR(C) + PR(D)$$

Contudo, geralmente os sites não apontam para um único local. Portanto, vamos supor que os sites $B$, $C$ e $D$ também tem conexões entre si.
<!-- spaces -->
<img src="img/ABCD3.png" width="300" height="300"/>
<!-- spaces -->

Observe que o site $B$ tem 3 conexões, enquanto o site $C$ tem 2 conexões e o site $D$ apenas 1. Voltemos a nosso hipotético usuário: caso ele se encontre no site $C$, pode tanto ir para $A$ quanto para $D$. Como ele não possui preferências, a chance para que vá para um desses sites é igual. Dessa forma, no cálculo do _PageRank_ de $A$, já não podemos considerar o valor total do _PageRank_ de $C$; e sim apenas metade. O mesmo ocorre para o valor proveniente do _PageRank_ de $B$. Nessa situação, o valor proveniente de $D$ permanece o mesmo pois possui apenas uma conexão.
Portanto, temos:

$$PR(A) = \displaystyle \frac{PR(B)}{3} + \frac{PR(C)}{2} + \frac{PR(D)}{1}$$

Podemos, então, generalizar essa conta: o _PageRank_ de um site vai ser a soma entre a razão do _PageRank_ de sites que referenciam o site original e o número de ligações contidos nesses outros sites. Assim, considerando o número de ligações do site $B$ o número $L(B)$ temos:
<!-- spaces -->
$$PR(A) = \displaystyle \frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)}$$
<!-- spaces -->
Para um número qualquer de sites, podemos utilizar a seguinte notação de somatória:
<!-- spaces -->
$${\displaystyle PR(u)=\sum _{v\in B_{u}}{\frac {PR(v)}{L(v)}}}$$
<!-- spaces -->
(sendo $B_u$ o conjunto de páginas que referenciam $u$)

### Erros

O sistema descrito é funcional para a maioria das situações. Porém, duas situações devem ser consideradas: a drenagem e o ciclo.


Voltemos na segunda imagem. Como descrito anteriormente, as ligações saindo de $A$ e as ligações de um site consigo mesmo foram ocultadas.
Vamos supor, entretanto, que as ligações saindo de $A$ não foram ocultadas, e sim que realmente não existem.
Dessa forma, o usuário hipotético entraria eventualmente em $A$ e seria incapaz de sair. Se analisarmos o _PageRank_, notaríamos que $A$ causou uma drenagem nos demais valores.
<!-- spaces -->
<img src="img/ABCD2.png" width="300" height="300"/>

A outra situação é quando temos um ciclo fechado, onde cada site possui uma unica ligação e eventualmente chegamos no site inicial.
<!-- spaces -->
<img src="img/ABCD4.png" width="300" height="300"/>
<!-- spaces -->
Nesse caso, temos que $A$ aponta somente para $B$, que por sua vez aponta somente para $C$, que aponta somente para $D$ e que por fim aponta somente para $A$. Assim, o nosso usuário hipotético fica preso nesse ciclo infinito.

### A solução

Para solucionar esses erros, adicionamos uma ideia em nosso algoritmo: a que o usuário hipotético pode cansar de ir para para outro site a partir de uma referência e, a qualquer momento, pode visitar qualquer site, sem preferência.
Portanto, denominamos de $d$ um fator de amortecimento: a probabilidade, contida entre 0 e 1, de que o usuário irá seguir a ideia inicial apresentada.

 Analogamente, temos que a probabilidade de que o usuário decida acessar qualquer site aleatoriamente será de $(1 - d)$. Como definimos que é aleatório, a chance para cada site nessa segunda escolha é equivalente, e portanto temos $\frac{(1 - d)}{N}$, sendo $N$ o número total de sites.  
Concluímos assim que o cálculo do _PageRank_ pode ser dado da seguinte maneira:
<!-- spaces -->
$$PR(p_i) = \displaystyle \frac{(1 - d)}{N} + d{\sum _{p_j\in M{p_i}}{\frac {PR(p_j)}{L(p_j)}}}$$
<!-- spaces -->
(com $M(p_i)$ sendo o conjunto de páginas que referenciam $p_i$)

### PageRank como um problema de Álgebra Linear

Vamos agora analisar como podemos utilizar de matrizes para facilitar o cálculo de nosso algoritmo.
O primeiro passo será transformar o cálculo genérico numa linguagem vetorial, para que tenhamos simultaneamente o cálculo de todos os _PageRanks_. 
Assim, denominamos $\mathbf {R}$ o vetor contendo o _PageRank_ dos sites $p_1$ até $p_n$.

Voltando ao nosso exemplo composto de 4 sites mais simples, no qual ainda não havia o fator de amortecimento, temos que $PR(A) = \frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)}$. Podemos reescrever esse segundo membro como o resultado de um produto entre o vetor linha  $\begin{bmatrix}\frac{1}{L(B)} & \frac{1}{L(C)} & \frac{1}{L(D)}\end{bmatrix}$ e o vetor coluna $\mathbf {R}$, o que novamente equivale a $PR(A)$. Agora, imagine que pegamos todos os vetores linhas para os demais sites e montamos uma matriz $\mathcal {M}$. O resultado do produto dessa matriz agora não seria mais $PR(A)$, e sim um vetor composto de todos os _PageRanks_: o vetor $\mathbf {R}$!

Porém, dois pontos devem ser levados em consideração: o primeiro, é que um site pode não ter referência para outro, e o segundo é que o fator de amortecimento está ausente da equação.
Para o primeiro ponto, vamos alterar a matriz $\mathcal {M}$ para que cada elemento seja uma função do tipo $\ell(p_i,p_j)$, que pode assumir dois valores: 0, caso não haja referência de $p_j$ para $p_i$, ou $\frac{1}{L(p_j)}$ caso haja.
Para o segundo ponto, basta multiplicar a matriz $\mathcal {M}$ pelo fator de amortecimento $d$ e adicionar um vetor coluna de tamanho n composto de elementos $\frac{(1-d)}{N}$
Assim, temos o seguinte resultado:
<!-- spaces -->

$${\displaystyle \mathbf {R} = \begin{bmatrix}PR(p_{1})\\PR(p_{2})\\\vdots \\PR(p_{N})\end{bmatrix}  ={\begin{bmatrix}{(1-d)/N}\\{(1-d)/N}\\\vdots \\{(1-d)/N}\end{bmatrix}}+d{\begin{bmatrix}\ell (p_{1},p_{1})&\ell (p_{1},p_{2})&\cdots &\ell (p_{1},p_{N})\\\ell (p_{2},p_{1})&\ddots &&\vdots \\\vdots &&\ell (p_{i},p_{j})&\\\ell (p_{N},p_{1})&\cdots &&\ell (p_{N},p_{N})\end{bmatrix}}\mathbf {R} }$$

Se tirarmos em evidência $\frac{(1-d)}{N}$ do primeiro vetor, temos um vetor composto apenas de 1. Podemos aproveitar o fato de que todos os _PageRanks_ somam 1 para reescrever esse vetor como o produto $\mathbf {ER}$, sendo $\mathbf {E}$ um matriz composta apenas de 1, ou seja:
<!-- spaces -->

$$\displaystyle \begin{bmatrix}{(1-d)/N}\\{(1-d)/N}\\\vdots \\{(1-d)/N}\end{bmatrix} = \frac{(1-d)}{N}\begin{bmatrix}{1}\\{1}\\\vdots \\{1}\end{bmatrix} = \frac{(1-d)}{N}ER$$

Dessa forma, todos os termos estão multiplicados pelo vetor coluna $R$, de forma que podemos mais uma vez tirar o fator em evidência:
<!-- spaces -->

$$\displaystyle \mathbf {R} = (\frac{(1-d)}{N}\mathbf {E} + d \mathcal {M}) \mathbf {R}$$

Se chamarmos $(\frac{(1-d)}{N}\mathbf {E} + d \mathcal {M})$ de $\widehat {\mathcal {M}}$, temos:
<!-- spaces -->
$$\displaystyle \mathbf {R} = \widehat {\mathcal {M}} \mathbf {R}$$
<!-- spaces -->
De onde tiramos que $\mathbf{R}$ **é um autovetor de** $\widehat {\mathcal {M}}$ **para o autovalor 1**.


Temos que os elementos da matriz $\widehat {\mathcal {M}}$ são não-negativos e que suas colunas somam 1 (ou em alguns casos raros, 0). Dessa forma, essa matriz é estotástica, ou seja, seu maior autovalor é 1. Conseguimos concluir assim que a matriz $\mathbf{R}$ de _PageRank_ é o **autovetor do maior autovalor** da matriz $\widehat {\mathcal {M}}$.

Portanto, um método para o cálculo de _PageRank_ é:
1. Descobrir os autovalores da matriz $\widehat {\mathcal {M}}$.
2. Encontrar, dentre eles, o maior.
3. Encontrar o autovetor para esse autovalor.
<!-- spaces -->

Contudo, esse método não é usual para matrizes muito grandes, pois demanda bastante poder computacional para descobrir todos seus autovalores.

### Método das potências

Podemos, contudo, utilizar das equações que encontramos para calcular um método de aproximar o vetor de _PageRank_ sem ter de calcular os autovalores da matriz.
Quando se tem uma matriz $A$ tal que um de seus autovalores é estritamente maior que os outros e temos um vetor $b_0$ que tem um componente não-nulo na direção do autovetor relacionado ao maior autovalores, temos a seguinte propriedade, denominada _Power Method_ ou _Power Iteration_:
<!-- spaces -->
$${\displaystyle b_{k+1}={\frac {Ab_{k}}{\|Ab_{k}\|}}}$$
<!-- spaces -->
Onde o vetor $b_0$ converge para o autovetor relacionado ao maior autovalor de $A$. Isso ocorre quando ele é multiplicado pela matriz, normalizado e o processo é repetido uma certa quantidade de vezes.  

Devido às propriedades da matriz $\widehat {\mathcal {M}}$ (estocástica, primitiva e irredutível), e sabendo que $R$ é justamente o autovetor relacionado ao maior autovalor dessa matriz, sabemos que podemos então utilizar esse método para aproximar o valor de $\mathbf{R}$ sem ter de calcular todos os autovalores.
<!-- spaces -->
Essas propriedades também fazem com que não seja necessário normalizar cada iteração, uma vez que esta etapa serve apenas para evitar um crescimento descontrolado do vetor.

Podemos iniciar com um vetor $\mathbf{R}_0$, onde as entradas são equivalentes; de valor $\frac{1}{N}$, sendo $N$ o número total de sites.
Assim, escrevemos:
<!-- spaces-->
$${\displaystyle \mathbf{R}_{k+1}={\widehat {\mathcal {M}}}\mathbf{R}_k}$$

Fazendo isso, podemos comparar a norma da diferença entre $\mathbf{R}_{k+1}$ e $\mathbf{R}_k$ até que ele seja menor que um valor escolhido muito pequeno.
<!-- spaces -->
Se tivermos $\displaystyle |\mathbf{R}_{k+1}-\mathbf{R}_k| \lt\epsilon$, então sabemos que os valores de $\mathbf{R}_{k+1}$ convergiram para valores muitos próximos ao do vetor $\mathbf{R}$ encontrado pelo método do autovetor.

Logo, encontramos os dois métodos que estaremos utilizando e comparando.